In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import copy

# A generative algorithm
We create a generative algorithm. Starting with a given configuration of the three particles, we consider every possible move of a single particle.
## initial configuration
We consider three particles : i, i+1, i+2, they belong to a square lattice, their positions (in lattice index) are denoted : $(x_i,y_i) (x_{i1},y_{i1}), (x_{i2},y_{i2})$. The excluded volume impose a minimum distance : 
$$(x_k-x_l) + (y_k-y_l) >= 2$$
## Moves
We move each particle according to one of the moving vectors defined as :
$$(1,0), (-1,0), (0,1), (0,-1) $$
We then check if the excluded volume is respected and append the resulting configuration to the list of possible moves if so
## 

In [16]:
def distance(xy0,xy1):
    return np.sqrt((xy0[0]-xy1[0])**2+(xy0[1]-xy1[1])**2+(xy0[2]-xy1[2])**2)
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper
moves_v = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0]])
distance_set = {0,2.,truncate(np.sqrt(2),3),truncate(np.sqrt(5),3)}#,3,truncate(2*2**0.5,3)}
class Configuration:
    """
    configuration class on which the moves are applied.
    -> the positions of the particles are stored in the referential
    of the i=0 one:
    the class contain the required checks :
    -> check_excluded : returns False if any particles violate
                        the excluded volume requirement
    -> check_distances : returns False if any distances violate
                         the bonds distances requirement
    """
    def __init__(self,position_vector=None,Nparticles = 3):
        self.Nparticles=Nparticles
        # initial configuration with Nparticles aligned separated by an empty site
        if type(position_vector) == np.ndarray:
            self.position_v = position_vector
        else:
            self.position_v = np.array([[0,2*i,0] for i in range(self.Nparticles)])
        self.initial_position = copy.copy(self.position_v)
    def get_tuple_position(self):
        return tuple(map(tuple, self.position_v))
    def reset_position(self):
        self.position_v = copy.copy(self.initial_position)
    def check_excluded(self):
        # check if no particle is within the neighboring of another one
        for xy0 in self.position_v:
            for xy1 in self.position_v:
                if (xy0 != xy1).any():
                    if sum(abs(xy0-xy1))<2:
                        return False
        return True
    def check_distances(self):
        for i in range(self.position_v.shape[0]-1):
            #for xy1 in self.position_v:
            xy0 = self.position_v[i]
            xy1 = self.position_v[i+1]
            if not truncate(distance(xy0,xy1),3) in distance_set:
                return False
        return True
    def apply_move(self,i,v):
        """
        apply the moving vector v to the ith particle
        then translate the system to have the 0th
        particle at point (0,0)
        """
        self.position_v[i] += v
        self.position_v-= self.position_v[0]
    def print_configuration(self):
        to_print = [['0 ' for _ in range(2*self.Nparticles+2)] for _ in range(2*self.Nparticles+2)]
        #to_print.fill(" 0 ")
        for ij in self.position_v:
            to_print[ij[0]-self.position_v[1,0]+3][ij[1]-self.position_v[1,1]+3] = "1 "
            #to_print[ij[0]+3][ij[1]+3] = "1 "
            # First row
        print(f"  ", end='')
        for j in range(8):
            print(f"| {j+1} ", end='')
        print(f"| ", end='')
        print()
        print(f'{(8*4+4)*"-"}')
        A=''
        for i in range(len(to_print)):
            print(f"{chr(65+i)} ", end='')
            for j in range(len(to_print[0])):
                if to_print[i][j]=='0 ':
                    print(f"| {' '} ", end='')
                else:
                    print(f"| {'1'} ", end='')
            print(f"| ", end='')
            print()
            print(f'{(8*4+4)*"-"}')
                #print(to_print[j][i],end='')
        print(A)

In [17]:
# Start with a configuration with three particles aligned
config = Configuration()
config.print_configuration()

  | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 
------------------------------------
A |   |   |   |   |   |   |   |   | 
------------------------------------
B |   |   |   |   |   |   |   |   | 
------------------------------------
C |   |   |   |   |   |   |   |   | 
------------------------------------
D |   | 1 |   | 1 |   | 1 |   |   | 
------------------------------------
E |   |   |   |   |   |   |   |   | 
------------------------------------
F |   |   |   |   |   |   |   |   | 
------------------------------------
G |   |   |   |   |   |   |   |   | 
------------------------------------
H |   |   |   |   |   |   |   |   | 
------------------------------------



### Start by generating all the possible configuration

In [18]:
# set that stores a unique sample of configuration
config_set_move1 = set()
config = Configuration()
# apply each vector moves to each particles:
#config_set_move1.add(config.get_tuple_position())
for v in moves_v:
    for p in range(config.Nparticles):
        config.reset_position()
        config.apply_move(p,v)
        #config.print_configuration()
        if not config.check_distances() or not config.check_excluded():
            continue
        config_set_move1.add(config.get_tuple_position())
print(config_set_move1.__len__())

6


In [19]:
#for position in list(config_set_move1):
#    print(position)
#    Configuration(np.array(position)).print_configuration()

In [20]:
# set that stores a unique sample of configuration
config_set_move2 = set()
config = Configuration()
# apply each vector moves to each particles:
for position in config_set_move1:
    config = Configuration(np.array(position))
    for v in moves_v:
        for p in range(config.Nparticles):
            config.reset_position()
            config.apply_move(p,v)
            #config.print_configuration()
            if not config.check_distances() or not config.check_excluded() or config.get_tuple_position in config_set_move1:
                continue
            config_set_move2.add(config.get_tuple_position())
print(config_set_move2.__len__())

17


In [7]:
#for position in list(config_set_move2):
#    print(position)
#    Configuration(np.array(position)).print_configuration()

### Compute all the possible position vector

In [21]:
# set that stores a unique sample of configuration
config_set_tot = set()
new_config_set = set()
config = Configuration()
# initialize the set with the first configuration
new_config_set.add(config.get_tuple_position())
# if all the position in new_config_set are already in config_set_top
# it means that we have all the configuration have been explored
while (new_config_set-new_config_set.intersection(config_set_tot)).__len__()!=0:
    config_set_tot |= new_config_set
    #print((new_config_set-new_config_set.intersection(config_set_tot)).__len__())
    previous_new_config_set = copy.copy(new_config_set)
    new_config_set = set()
    for position in previous_new_config_set:
        config = Configuration(np.array(position))
        for v in moves_v:
            for p in range(config.Nparticles):
                config.reset_position()
                config.apply_move(p,v)
                #config.print_configuration()
                if not config.check_distances() or not config.check_excluded():
                    continue
                new_config_set.add(config.get_tuple_position())
print(config_set_tot.__len__())

208


In [22]:
def test_if_middle_particle_moved(position1,position2):
    """
    Given two position vector, this function test if it is possible
    to go from one to the other by just moving the middle particle.
    """
    if position1[0]!=position2[0] or position1[2]!=position2[2]:
        return False
    if abs(position1[1][0]-position2[1][0])+abs(position1[1][1]-position2[1][1])==1:
        #print(np.append(np.array(position1),np.array([position2[1]]),axis=0))
        #print(position1,position2)
        return True
    return False

### Compute all the processes coordinates for the middle particles

In [23]:
Coordinates_middle = list()
for position1 in config_set_tot:
    #old_size = Coordinates_middle.__len__()
    for position2 in config_set_tot:
        if test_if_middle_particle_moved(position1,position2):
            Coordinates_middle.append(np.append(np.array(position1),np.array([position2[1]]),axis=0))
    #if Coordinates_middle.__len__()-old_size==0:
    #    config = Configuration(np.array(position1))
    #    config.print_configuration()
Coordinates_middle=np.array(Coordinates_middle)
print(Coordinates_middle.shape)

(208, 4, 3)


## Compute all the processes coordinate for the side particles
### Start by computing all the possible 2 particles configuration

In [24]:
# set that stores a unique sample of configuration
config_set_tot2 = set()
new_config_set = set()
config = Configuration(Nparticles=2)
# initialize the set with the first configuration
new_config_set.add(config.get_tuple_position())
# if all the position in new_config_set are already in config_set_top
# it means that we have all the configuration have been explored
while (new_config_set-new_config_set.intersection(config_set_tot2)).__len__()!=0:
    config_set_tot2 |= new_config_set
    #print((new_config_set-new_config_set.intersection(config_set_tot)).__len__())
    previous_new_config_set = copy.copy(new_config_set)
    new_config_set = set()
    for position in previous_new_config_set:
        config = Configuration(np.array(position),Nparticles=2)
        for v in moves_v:
            for p in range(config.Nparticles):
                config.reset_position()
                config.apply_move(p,v)
                #config.print_configuration()
                if not config.check_distances() or not config.check_excluded():
                    continue
                new_config_set.add(config.get_tuple_position())
print(config_set_tot2.__len__())

16


In [25]:
def test_if_2_particle_moved(position1,position2):
    """
    Given two position vector, this function test if it is possible
    to go from one to the other by just moving the middle particle.
    """
    if abs(position1[1][0]-position2[1][0])+abs(position1[1][1]-position2[1][1])==1:
        #print(np.append(np.array(position1),np.array([position2[1]]),axis=0))
        return True
    return False

In [26]:
Coordinates_extrem = list()
for position1 in config_set_tot2:
    #old_size = Coordinates_middle.__len__()
    for position2 in config_set_tot2:
        if test_if_2_particle_moved(position1,position2):
            Coordinates_extrem.append(np.append(np.array(position1),np.array([position2[1]]),axis=0))
    #if Coordinates_middle.__len__()-old_size==0:
    #    config = Configuration(np.array(position1))
    #    config.print_configuration()
Coordinates_extrem = np.array(Coordinates_extrem)
print(Coordinates_extrem.shape)

(32, 3, 3)
